## Loading Libraries

- We need Two important libraries for this project:
- [ ] hazm
- [ ] parsivar
- I use parsivar for Normalization, Tokenization and Stemming and use hazm for find stopwords and remove them
- Also we use pandas for working with dataframe and also numpy for working with arrays

In [4]:
from parsivar import Normalizer, Tokenizer, FindStems
from hazm import stopwords_list
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import json
import pickle
import re
import math
import numpy as np
import matplotlib.pyplot as plt
from string import punctuation
from sklearn.linear_model import LinearRegression
import string
import pandas as pd

## Loading Dataset

In [10]:
# If we work with google colab
"""
from google.colab import drive
drive.mount('/content/drive')
data_path = '/content/drive/MyDrive/IR_data_news_12k.json'
df = pd.read_json(data_path)
"""
# If we work on local system
df = pd.read_json('Data/IR_data_news_12k.json')

## EDA

- First We should get some information about our dataset. If we konw it well we can do better

In [11]:
df.head(10)

,0,1,2,3,4,5,6,7,8,9,...,12192,12193,12194,12195,12196,12197,12198,12199,12200,12201
title,اعلام زمان قرعه کشی جام باشگاه های فوتسال آسیا,سجادی :حضور تماشاگران در لیگ برتر فوتبال تابع...,محل برگزاری نشست‌های خبری سرخابی‌ها؛ مجیدی در ...,ماجدی در نشست با صالحی امیری: امیدوارم در این ...,لیگ‌برتر بسکتبال|‌ نخستین پیروزی شهرداری گرگان...,مسابقات تنیس روی میز فیدر قطر| هر4 بانوی ملی پ...,اعلام برنامه نشست خبری گل محمدی/ مجیدی هم باید...,احضار مدیران پرسپولیس به کمیته انضباطی پیش از...,مدیر بی‌دستاورد، رئیس شد/ روزگار تلخ‌تر از تلخ...,خبر خوب برای استقلال؛ دانشگر با تیم تمرین کرد,...,کمیته ویژه‌ای از سوی رئیس جمهور مسئول پیگیری م...,مصری: با سفرهای استانی، کُلاه سر رئیس‌جمهور نم...,رئیسی به اردبیل سفر می‌کند/ توضیحاتی درباره مق...,رئیس دفتر رئیس جمهور وضعیت زلزله زرند را بررسی...,جزئیات گزارش سفر نظارتی اعضای کمیسیون فرهنگی م...,دفع حمله دزدان دریایی به دو نفتکش ایرانی توسط ...,نقدی بر یادداشت «مرزبندی گفتمانی با طالبان»/ و...,روند تحقیق و تفحص از سازمان سنجش با جدیت ادامه...,محسن اسلامی مدیرکل دفتر امور سیاسی وزارت کشور شد,جلسه سران قوا به‌ میزبانی رئیس مجلس برگزار شد
content,\nبه گزارش خبرگزاری فارس، کنفدراسیون فوتبال آس...,\nبه گزارش خبرگزاری فارس، سید حمید سجادی در حا...,\nبه گزارش خبرگزاری فارس، نشست خبری پیش از مسا...,\nبه گزارش خبرگزاری فارس، سید رضا صالحی امیری...,\nبه گزارش خبرنگار ورزشی خبرگزاری فارس، در نخس...,\nبه گزارش خبرگزاری فارس، چهار بانوی تنیس روی ...,\nبه گزارش خبرگزاری فارس و به نقل از سایت باش...,\nبه گزارش خبرنگار ورزشی خبرگزاری فارس، کمیته ...,\nبه گزارش خبرنگار ورزشی خبرگزاری فارس، مجمع ا...,\nبه گزارش خبرنگار ورزشی خبرگزاری فارس، تمرین ...,...,\nاسماعیلی حسین زهی نماینده مردم خاش در مجلس ش...,\nعبدالرضا مصری نایب رئیس مجلس شورای اسلامی در...,\nبه گزارش گروه سیاسی خبرگزاری فارس، سیّد صولت...,\nبه گزارش گروه سیاسی خبرگزاری فارس غلامحسین ا...,\nغلامرضا منتظری نماینده مردم گرگان در مجلس شو...,\nبه گزارش گروه دفاعی خبرگزاری فارس، دریادار ش...,\nخبرگزاری فارس ـ یادداشت میهمان ـ مسعود مرادی...,\nبه گزارش خبرنگار پارلمانی خبرگزاری فارس، هیا...,\nبه گزارش خبرگزاری فارس به نقل از پایگاه اطلا...,\nبه گزارش خبرنگار پارلمانی خبرگزاری فارس، جلس...
tags,"[اعلام زمان, قرعه‌کشی, قرعه‌کشی جام, قرعه‌کشی ...","[سجادی, لیگ, فدراسیون, وزیر ورزش]","[دربی 94, محل برگزاری, خبری سرخابی‌ها, مجیدی, ...","[کمیته امداد امام خمینی (ره), کمیته ملی المپیک...","[بسکتبال, لیگ برتر بسکتبال, شهرداری گرگان, تیم...","[مهشید اشتری, تنیس روی میز, ملی پوشان تنیس روی...","[باشگاه استقلال, یحیی گل محمدی, فرهاد مجیدی, ن...","[باشگاه پرسپولیس, احضار مدیران پرسپولیس به کمی...","[والیبال, پیمان رضایی, تیم والیبال شهرداری ارو...","[دانشگر با تیم تمرین کرد, محمد دانشگر, استقلال]",...,"[مجلس شورای اسلامی, کمیسیون عمران, اسماعیل حسی...","[سید ابراهیم رئیسی, مجلس شورای اسلامی, رئیس‌جم...","[سید ابراهیم رئیسی, صولت مرتضوی, سفر رئیسی به ...","[غلامحسین اسماعیلی, رئیس دفتر رئیس جمهور, زلزل...","[مجلس شورای اسلامی, غلامرضا منتظری, نایب رئیس ...","[ارتش جمهوری اسلامی ایران, شهرام ایرانی, نیروی...","[طالبان, یادداشت, گفتمان, جمهوری اسلامی ایران,...","[مجلس شورای اسلامی, سازمان سنجش و آموزش کشور, ...","[وزارت کشور, احمد وحیدی, وزیر کشور, وزارت علوم...","[سران قوا, رئیس مجلس, رئیس جمهور, رئیس قوه قضا..."
date,3/15/2022 5:59:27 PM,3/15/2022 5:30:07 PM,3/15/2022 5:20:01 PM,3/15/2022 5:18:00 PM,3/15/2022 5:16:41 PM,3/15/2022 5:15:34 PM,3/15/2022 3:57:47 PM,3/15/2022 3:43:18 PM,3/15/2022 3:35:21 PM,3/15/2022 3:21:04 PM,...,10/17/2021 7:17:00 AM,10/17/2021 6:05:00 AM,10/16/2021 8:58:18 PM,10/16/2021 8:47:20 PM,10/16/2021 7:38:13 PM,10/16/2021 6:25:36 PM,10/16/2021 5:03:39 PM,10/16/2021 4:54:47 PM,10/16/2021 4:14:30 PM,10/16/2021 4:05:31 PM
url,https://www.farsnews.ir/news/14001224001005/اع...,https://www.farsnews.ir/news/14001224000982/سج...,https://www.farsnews.ir/news/14001224000971/مح...,https://www.farsnews.ir/news/14001224000964/ما...,https://www.farsnews.ir/news/14001224000947/لی...,https://www.farsnews.ir/news/14001224000916/مس...,https://www.farsnews.ir/news/14001224000865/اع...,https://www.farsnews.ir/news/14001224000842/اح...,https://www.farsnews.ir/news/140012

So it has 6 rows and 12202 columns. but this is unusual and we should transpose it.

In [12]:
df = df.transpose()
df.head(5)

,title,content,tags,date,url,category
0,اعلام زمان قرعه کشی جام باشگاه های فوتسال آسیا,\nبه گزارش خبرگزاری فارس، کنفدراسیون فوتبال آس...,"[اعلام زمان, قرعه‌کشی, قرعه‌کشی جام, قرعه‌کشی ...",3/15/2022 5:59:27 PM,https://www.farsnews.ir/news/14001224001005/اع...,sports
1,سجادی :حضور تماشاگران در لیگ برتر فوتبال تابع...,\nبه گزارش خبرگزاری فارس، سید حمید سجادی در حا...,"[سجادی, لیگ, فدراسیون, وزیر ورزش]",3/15/2022 5:30:07 PM,https://www.farsnews.ir/news/14001224000982/سج...,sports
2,محل برگزاری نشست‌های خبری سرخابی‌ها؛ مجیدی در ...,\nبه گزارش خبرگزاری فارس، نشست خبری پیش از مسا...,"[دربی 94, محل برگزاری, خبری سرخابی‌ها, مجیدی, ...",3/15/2022 5:20:01 PM,https://www.farsnews.ir/news/14001224000971/مح...,sports
3,ماجدی در نشست با صالحی امیری: امیدوارم در این ...,\nبه گزارش خبرگزاری فارس، سید رضا صالحی امیری...,"[کمیته امداد امام خمینی (ره), کمیته ملی المپیک...",3/15/2022 5:18:00 PM,https://www.farsnews.ir/news/14001224000964/ما...,sports
4,لیگ‌برتر بسکتبال|‌ نخستین پیروزی شهرداری گرگان...,\nبه گزارش خبرنگار ورزشی خبرگزاری فارس، در نخس...,"[بسکتبال, لیگ برتر بسکتبال, شهرداری گرگان, تیم...",3/15/2022 5:16:41 PM,https://www.farsnews.ir/news/14001224000947/لی...,sports


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12202 entries, 0 to 12201
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   title     12202 non-null  object
 1   content   12202 non-null  object
 2   tags      12202 non-null  object
 3   date      12202 non-null  object
 4   url       12202 non-null  object
 5   category  12202 non-null  object
dtypes: object(6)
memory usage: 925.3+ KB


Thats very good. we dont have any null object. 

In [14]:
df['category'].unique()

array(['sports', 'politics'], dtype=object)

According to category column we only have two class of news : Sports and politics

In [15]:
len(df['date'].unique())

12183

# Step1)
## Data Preprocessing

- First we should define some functions for removing punctuations and stemming and removing stopwords. because for doing these preprocessing jobs we dont have a straightforward function in library.

In [16]:
def remove_punc (input_content):
    return re.sub(r'[^\w\s]','',input_content)

def stemmer (tokens):
    stemmed = []
    st = FindStems()
    for i in tokens:
        stemmed_token = st.convert_to_stem(i)
        stemmed.append(stemmed_token)
    return stemmed

def stopwords_remover(stemmed):
    stop_words = stopwords_list()
    
    for i in stemmed:
        if i in stop_words:
            stemmed.remove(i)
    return stemmed

- Now we define a function for single news preprocessing and showing each steps

In [17]:
def single_preprocess (input_content):
    print("Original content:")
    print(input_content+"\n")
    # 1) first we remove punctuations
    punc_removed = remove_punc(input_content)
    print("After removing puncs:")
    print(punc_removed+"\n")
    # 2) second we Normalize it
    normal_content = Normalizer().normalize(punc_removed)
    print("After normalizing:")
    print(normal_content+"\n")
    # 3) then we can tokenize content
    tokened_content = Tokenizer().tokenize_words(normal_content)
    print("After tokenizing:")
    print(tokened_content,'\n')
    # 4) then we remove stopwords
    # (Attention: in the parsivar library there isn't any functions to remove stopwords. so we use hazm for this part)
    removed_content = stopwords_remover(tokened_content)
    print("After removing stopwords:")
    print(removed_content,"\n")
    # 5) then we stemming
    stemmed_content = stemmer(removed_content)
    print("After stemming:")
    print(stemmed_content,"\n")